In [1]:
import clingo
from gamearg import *
from IPython.display import display, Image
import pandas as pd
import concurrent
from concurrent.futures import ThreadPoolExecutor
import time

## Generating Graph 

With given `node_num`, program will be able to generate all possible edges

In [2]:
def graph_generating(node_num):
    def on_model(m):
        if str(m) == "":
            possible_graphs.append(str(m))
        else:
            elements = str(m).split(" ")
            elements_with_dot = [element + "." for element in elements]
            concatenated = " ".join(elements_with_dot)
            possible_graphs.append(concatenated)

    graph_log = """
    #const n = {}.
    node(1..n).
    {{move(X,Y): node(X), node(Y)}}.
    edge(X,Y):- move(X,Y).
    edge(X,Y):- move(Y,X).
    connected(X,Y) :- edge(X,Y).
    connected(X,Y) :- connected(X,Z), edge(Z,Y).
    :- node(X), node(Y), not connected(X,Y), X!=Y.
    #show move/2.
    """.format(node_num)
    # print(graph_log)
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.add("base", [], graph_log)
    ctl.ground([("base", [])])
    possible_graphs = []
    ctl.solve(on_model=on_model)
    return possible_graphs

In [3]:
possible_graphs = graph_generating(node_num=2)

In [4]:
possible_graphs

['move(2,1).',
 'move(2,1). move(2,2).',
 'move(1,1). move(2,1).',
 'move(1,1). move(2,1). move(2,2).',
 'move(1,2).',
 'move(1,1). move(1,2).',
 'move(1,2). move(2,2).',
 'move(1,1). move(1,2). move(2,2).',
 'move(1,2). move(2,1).',
 'move(1,2). move(2,1). move(2,2).',
 'move(1,1). move(1,2). move(2,1).',
 'move(1,1). move(1,2). move(2,1). move(2,2).']

## Digest into Game Solver

Based on the `possible_graphs`, we need to digest them into game_solver one by one and ensure that the result will be 3-valued (dir=back and forward)

In [5]:
columns = [
    "node_number",
    "edge_number",
    "graph",
    "fw_node_wfs",
    "bw_node_wfs",
]
graph_cal = pd.DataFrame(columns=columns)
node_num = 1


def process_graph(graph, node_num):
    try:
        fw_node_wfs_df = node_wfs_cal(graph, "game", reverse=False)
        bw_node_wfs_df = node_wfs_cal(graph, "game", reverse=True)
        return {
            "node_number": node_num,
            "edge_number": len(graph.split(".")) - 1,
            "graph": graph,
            "fw_node_wfs": set(fw_node_wfs_df["wfs"]),
            "bw_node_wfs": set(bw_node_wfs_df["wfs"]),
        }
    except Exception as e:
        print(f"An error occurred while processing {graph}: {e}")
        return None


data_to_add = []
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for node_num in range(1, 5):
        possible_graphs = graph_generating(node_num)
        if not possible_graphs or all(graph == "" for graph in possible_graphs):
            continue
        for graph in possible_graphs:
            if graph == "":
                continue
            else:
                futures.append(executor.submit(process_graph, graph, node_num))
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            data_to_add.append(result)
graph_cal = pd.DataFrame(data_to_add, columns=columns)

In [6]:
graph_cal

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs
0,3,4,"move(1,3). move(2,1). move(2,2). move(2,3).","{won, lost}",{draw}
1,3,4,"move(2,1). move(2,3). move(3,1). move(3,2).","{won, lost}",{draw}
2,3,5,"move(1,1). move(1,3). move(2,1). move(2,3). mo...",{draw},"{won, lost}"
3,3,6,"move(1,1). move(2,1). move(2,2). move(2,3). mo...",{draw},{draw}
4,3,4,"move(1,1). move(1,3). move(2,1). move(2,3).","{won, lost}","{won, lost}"
...,...,...,...,...,...
61784,4,9,"move(1,1). move(1,4). move(2,2). move(2,4). mo...",{draw},{draw}
61785,4,8,"move(1,4). move(2,2). move(2,4). move(3,3). mo...",{draw},{draw}
61786,4,9,"move(1,1). move(1,4). move(2,2). move(2,4). mo...",{draw},{draw}
61787,4,9,"move(1,4). move(2,2). move(2,4). move(3,3). mo...",{draw},{draw}


In [7]:
graph_cal["fw_len"] = graph_cal["fw_node_wfs"].apply(len)
graph_cal["bw_len"] = graph_cal["bw_node_wfs"].apply(len)

In [8]:
graph_cal[(graph_cal["fw_len"] == 3) & (graph_cal["bw_len"] == 3)].sort_values(
    ["node_number", "edge_number"]
)

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,fw_len,bw_len
142,4,4,"move(1,2). move(1,3). move(3,4). move(4,4).","{won, draw, lost}","{won, draw, lost}",3,3
517,4,4,"move(1,2). move(2,2). move(3,1). move(3,4).","{won, draw, lost}","{won, draw, lost}",3,3
593,4,4,"move(2,1). move(3,1). move(4,3). move(4,4).","{won, draw, lost}","{won, draw, lost}",3,3
638,4,4,"move(1,3). move(2,1). move(2,2). move(4,3).","{won, draw, lost}","{won, draw, lost}",3,3
895,4,4,"move(1,4). move(2,1). move(3,1). move(3,3).","{won, draw, lost}","{won, draw, lost}",3,3
...,...,...,...,...,...,...,...
57632,4,7,"move(1,1). move(1,4). move(3,1). move(3,2). mo...","{won, draw, lost}","{won, draw, lost}",3,3
58134,4,7,"move(2,2). move(2,4). move(3,1). move(3,4). mo...","{won, draw, lost}","{won, draw, lost}",3,3
58368,4,7,"move(1,3). move(1,4). move(2,2). move(2,4). mo...","{won, draw, lost}","{won, draw, lost}",3,3
60546,4,7,"move(1,1). move(1,4). move(2,3). move(2,4). mo...","{won, draw, lost}","{won, draw, lost}",3,3


In [11]:
graph_cal[
    (graph_cal["fw_len"] == 3)
    & (graph_cal["bw_len"] == 3)
    & (graph_cal["node_number"] == 4)
    & (graph_cal["edge_number"] == 4)
].sort_values(["node_number", "edge_number"])

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,fw_len,bw_len
142,4,4,"move(1,2). move(1,3). move(3,4). move(4,4).","{won, draw, lost}","{won, draw, lost}",3,3
517,4,4,"move(1,2). move(2,2). move(3,1). move(3,4).","{won, draw, lost}","{won, draw, lost}",3,3
593,4,4,"move(2,1). move(3,1). move(4,3). move(4,4).","{won, draw, lost}","{won, draw, lost}",3,3
638,4,4,"move(1,3). move(2,1). move(2,2). move(4,3).","{won, draw, lost}","{won, draw, lost}",3,3
895,4,4,"move(1,4). move(2,1). move(3,1). move(3,3).","{won, draw, lost}","{won, draw, lost}",3,3
...,...,...,...,...,...,...,...
61509,4,4,"move(1,1). move(1,4). move(3,4). move(4,2).","{won, draw, lost}","{won, draw, lost}",3,3
61549,4,4,"move(1,4). move(3,3). move(4,2). move(4,3).","{won, draw, lost}","{won, draw, lost}",3,3
61551,4,4,"move(1,4). move(2,2). move(4,2). move(4,3).","{won, draw, lost}","{won, draw, lost}",3,3
61567,4,4,"move(1,4). move(2,2). move(2,4). move(4,3).","{won, draw, lost}","{won, draw, lost}",3,3


In [10]:
# graph_cal.to_csv("graph_cal_selfloop.csv")